In [57]:
# ==============================================
# model_training.ipynb — Final Clean & App-Aligned Version
# ==============================================

# --- Imports and setup ---
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch_geometric.nn import SAGEConv, GCNConv, GATConv
from torch_geometric.utils import from_networkx
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import pickle

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("✅ Using device:", device)

# Paths
DATA_DIR = "../data"
MODELS_DIR = "../models"
os.makedirs(MODELS_DIR, exist_ok=True)

# --- Load feature datasets ---
df_ig = pd.read_csv(os.path.join(DATA_DIR, "instagram_features.csv"))
df_fb = pd.read_csv(os.path.join(DATA_DIR, "facebook_features.csv"))

print(f"✅ Instagram data: {df_ig.shape}")
print(f"✅ Facebook data: {df_fb.shape}")

# --- Define cleaning helper ---
def clean_dataset(df):
    df = df.replace(["#DIV/0!", "N/A", "na", "NaN", "None", " "], np.nan)
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.fillna(0)
    return df

# --- Prepare features and labels ---
ig_features = [c for c in df_ig.columns if c != "label"]
fb_features = [c for c in df_fb.columns if c != "label"]

# Clean datasets
df_ig[ig_features] = clean_dataset(df_ig[ig_features])
df_fb[fb_features] = clean_dataset(df_fb[fb_features])

# Scale features
scaler_ig = StandardScaler()
X_ig = scaler_ig.fit_transform(df_ig[ig_features])
y_ig = df_ig["label"].astype(int).values

scaler_fb = StandardScaler()
X_fb = scaler_fb.fit_transform(df_fb[fb_features])
y_fb = df_fb["label"].astype(int).values

# Save individual scalers
with open(os.path.join(MODELS_DIR, "scaler_instagram.pkl"), "wb") as f:
    pickle.dump(scaler_ig, f)
with open(os.path.join(MODELS_DIR, "scaler_facebook.pkl"), "wb") as f:
    pickle.dump(scaler_fb, f)

print("✅ Scalers saved successfully!")

# --- Build kNN graphs ---
def build_knn_graph(X, k=10):
    A = kneighbors_graph(X, n_neighbors=k, include_self=False, mode='connectivity')
    G = nx.from_scipy_sparse_array(A)
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['x'] = X[i]
    return G

G_ig = build_knn_graph(X_ig, k=10)
G_fb = build_knn_graph(X_fb, k=10)

for i, n in enumerate(G_ig.nodes()):
    G_ig.nodes[n]["y"] = int(y_ig[i])
for i, n in enumerate(G_fb.nodes()):
    G_fb.nodes[n]["y"] = int(y_fb[i])

data_ig = from_networkx(G_ig)
data_fb = from_networkx(G_fb)

data_ig.x = torch.tensor(np.vstack([data_ig.x[i] for i in range(len(data_ig.x))]), dtype=torch.float)
data_ig.y = torch.tensor([int(v) for v in data_ig.y], dtype=torch.long)

data_fb.x = torch.tensor(np.vstack([data_fb.x[i] for i in range(len(data_fb.x))]), dtype=torch.float)
data_fb.y = torch.tensor([int(v) for v in data_fb.y], dtype=torch.long)

print("✅ Graphs built for both platforms!")

# --- Train/test masks ---
def create_masks(data, test_size=0.2):
    n = data.num_nodes
    idx = np.arange(n)
    train_idx, test_idx = train_test_split(idx, test_size=test_size, stratify=data.y.numpy())
    train_mask = torch.zeros(n, dtype=torch.bool)
    test_mask = torch.zeros(n, dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True
    return train_mask, test_mask

data_ig.train_mask, data_ig.test_mask = create_masks(data_ig)
data_fb.train_mask, data_fb.test_mask = create_masks(data_fb)
print("✅ Train/test masks created successfully!")

# --- Define Models ---
class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, out_channels=2, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = dropout
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

class GCNNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, out_channels=2, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=32, out_channels=2, heads=2, dropout=0.5):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)
        self.dropout = dropout
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# --- Evaluation & Training ---
def evaluate(logits, labels, mask):
    probs = F.softmax(logits, dim=1)[:,1].cpu().numpy()
    preds = logits.argmax(dim=1).cpu().numpy()
    y_true = labels.cpu().numpy()
    mask = mask.cpu().numpy()
    y_true_m, preds_m, probs_m = y_true[mask], preds[mask], probs[mask]
    return {
        "accuracy": accuracy_score(y_true_m, preds_m),
        "precision": precision_score(y_true_m, preds_m, zero_division=0),
        "recall": recall_score(y_true_m, preds_m, zero_division=0),
        "f1": f1_score(y_true_m, preds_m, zero_division=0),
        "roc_auc": roc_auc_score(y_true_m, probs_m) if len(np.unique(y_true_m))>1 else float('nan')
    }

def train_gnn(model, data, epochs=80, lr=1e-3, wd=5e-4, device="cpu"):
    model = model.to(device)
    data = data.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for epoch in range(1, epochs+1):
        model.train()
        opt.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        opt.step()
        if epoch % 10 == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                logits = model(data.x, data.edge_index)
                metrics = evaluate(logits, data.y, data.test_mask)
            print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | F1: {metrics['f1']:.4f} | Acc: {metrics['accuracy']:.4f}")
    return model, metrics

# --- Train and Save All Models ---
summary = []

for platform, data in [("Instagram", data_ig), ("Facebook", data_fb)]:
    in_dim = data.num_node_features
    print(f"\n🚀 Training Models for {platform} (features={in_dim})")

    models = {
        "GraphSAGE": GraphSAGENet(in_dim),
        "GCN": GCNNet(in_dim),
        "GAT": GATNet(in_dim)
    }

    for name, model in models.items():
        print(f"\n⚙️ Training {name}...")
        trained_model, metrics = train_gnn(model, data, device=device)
        summary.append({
            "Platform": platform,
            "Model": name,
            **metrics
        })
        torch.save(trained_model.state_dict(),
                   os.path.join(MODELS_DIR, f"{platform.lower()}_{name.lower()}.pt"))
        print(f"✅ Saved {name} model for {platform}")

# --- Save Summary ---
df_summary = pd.DataFrame(summary)
print("\n📊 Model Performance Comparison:")
display(df_summary)

df_summary.to_csv(os.path.join(MODELS_DIR, "training_summary.csv"), index=False)
print(f"✅ Summary saved to {os.path.join(MODELS_DIR, 'training_summary.csv')}")


✅ Using device: cpu
✅ Instagram data: (7500, 12)
✅ Facebook data: (206, 10)
✅ Scalers saved successfully!
✅ Graphs built for both platforms!
✅ Train/test masks created successfully!

🚀 Training Models for Instagram (features=11)

⚙️ Training GraphSAGE...
Epoch 001 | Loss: 0.7764 | F1: 0.5768 | Acc: 0.4473
Epoch 010 | Loss: 0.5770 | F1: 0.8254 | Acc: 0.8000
Epoch 020 | Loss: 0.4163 | F1: 0.8730 | Acc: 0.8627
Epoch 030 | Loss: 0.3199 | F1: 0.8939 | Acc: 0.8893
Epoch 040 | Loss: 0.2528 | F1: 0.9153 | Acc: 0.9133
Epoch 050 | Loss: 0.2122 | F1: 0.9301 | Acc: 0.9293
Epoch 060 | Loss: 0.1833 | F1: 0.9396 | Acc: 0.9393
Epoch 070 | Loss: 0.1579 | F1: 0.9428 | Acc: 0.9427
Epoch 080 | Loss: 0.1412 | F1: 0.9493 | Acc: 0.9493
✅ Saved GraphSAGE model for Instagram

⚙️ Training GCN...
Epoch 001 | Loss: 0.7400 | F1: 0.5352 | Acc: 0.4627
Epoch 010 | Loss: 0.6162 | F1: 0.7390 | Acc: 0.6987
Epoch 020 | Loss: 0.5063 | F1: 0.8333 | Acc: 0.8173
Epoch 030 | Loss: 0.4272 | F1: 0.8836 | Acc: 0.8793
Epoch 040 |

Platform      Model  accuracy  precision    recall        f1   roc_auc
0  Instagram  GraphSAGE  0.949333   0.950535  0.948000  0.949266  0.990757
1  Instagram        GCN  0.937333   0.936170  0.938667  0.937417  0.986491
2  Instagram        GAT  0.935333   0.931308  0.940000  0.935634  0.988199
3   Facebook  GraphSAGE  1.000000   1.000000  1.000000  1.000000  1.000000
4   Facebook        GCN  1.000000   1.000000  1.000000  1.000000  1.000000
5   Facebook        GAT  1.000000   1.000000  1.000000  1.000000  1.000000

✅ Summary saved to ../models\training_summary.csv


In [58]:
import pickle

with open("../rag_index/kb_meta.pkl", "rb") as f:
    meta = pickle.load(f)

for m in meta:
    if "instagram" in m.get("platform", "").lower():
        if m["label"] == "Fake":
            m["label"] = "Real"
        elif m["label"] == "Real":
            m["label"] = "Fake"

with open("../rag_index/kb_meta.pkl", "wb") as f:
    pickle.dump(meta, f)

print("✅ Corrected Instagram labels in kb_meta.pkl")


✅ Corrected Instagram labels in kb_meta.pkl
